# Using HuggingFace to train a model

In this notebook, we will be using `distilbert-base-uncased-finetuned-sst-2-english` to train a text classification model.

## Loading the data

Firstly, we need to merge the translated dataset with the original one.

In [15]:
import pandas as pd

dataset = pd.read_excel("../data/OpArticles_ADUs.xlsx")
translated_dataset = pd.read_csv("../data/translated_spans.csv")

dataset

,article_id,annotator,node,ranges,tokens,label
0,5d04a31b896a7fea069ef06f,A,0,"[[2516, 2556]]",O facto não é apenas fruto da ignorância,Value
1,5d04a31b896a7fea069ef06f,A,1,"[[2568, 2806]]",havia no seu humor mais jornalismo (mais inves...,Value
2,5d04a31b896a7fea069ef06f,A,3,"[[3169, 3190]]",É tudo cómico na FIFA,Value
3,5d04a31b896a7fea069ef06f,A,4,"[[3198, 3285]]",o que todos nós permitimos que esta organizaçã...,Value
4,5d04a31b896a7fea069ef06f,A,6,"[[4257, 4296]]",não nos fazem rir à custa dos poderosos,Value
...,...,...,...,...,...,...
16738,5cf4b764896a7fea06032673,D,29,"[[4980, 5041], [5074, 5279]]",A única variável disponibilizada que pode ser ...,Value
16739,5cf4b764896a7fea06032673,D,30,"[[5293, 5340]]",esse número esconde informação muito pertinente,Fact
16740,5cf4b764896a7fea06032673,D,32,"[[5053, 5072]]",bastante imperfeita,Value(-)
16741,5cf4b764896a7fea06032673,D,34,"[[5549, 5643]]",esconde também a proporção de diplomados que e...,Value


In [16]:
import numpy as np

grouped_df = dataset.groupby(by=['article_id', 'ranges'])
dataset_dict = {"tokens": [], "label": []}

for i, group in grouped_df:
    dict_counts = {x: group["label"].value_counts()[x] for x in np.unique(group[['label']].values)}
    if len(dict_counts.keys()) > 1:
        continue
    #dataset_dict["article_id"].append(group["article_id"].values[0])
    dataset_dict["tokens"].append(group["tokens"].values[0])
    dataset_dict["label"].append(list(dict_counts.keys())[0])
    
dataset = pd.DataFrame(dataset_dict, columns = ["tokens", "label", "article_id"])
dataset

,tokens,label,article_id
0,presumo que essas partilhas tenham gerado um e...,Value,NaN
1,essas partilhas tenham gerado um efeito bola d...,Value,NaN
2,esta questão ter [justificadamente] despertado...,Value,NaN
3,a ocasião propicia um debate amplo na sociedad...,Value,NaN
4,a tomada urgente de medidas por parte da tutel...,Value,NaN
...,...,...,...
10248,Um presidente de câmara pode pertencer à admin...,Value,NaN
10249,eticamente é reprovável,Value(-),NaN
10250,"eticamente é reprovável e, o bom senso, aconse...",Value,NaN
10251,"o bom senso, aconselha a não o fazer",Value,NaN


In [17]:
#dataset = dataset.join(translated_dataset)
dataset["label"].value_counts()

Value       5003
Fact        2235
Value(-)    1768
Value(+)     849
Policy       398
Name: label, dtype: int64

### Converting it into a Hugging Face dataset

In [18]:
!pip install datasets --quiet
from datasets import Dataset

labels = ['Value','Fact','Value(+)','Value(-)','Policy']
numeric_labels = []

for label in dataset["label"].values:
    new_label = labels.index(label)
    numeric_labels.append(new_label)

dataset["label"] = numeric_labels

dataset_hf = Dataset.from_pandas(dataset)

In [36]:
dataset["tokens"] = translated_dataset["Translated Spans"]

In [37]:
dataset

,tokens,label,article_id,translated
0,I assume these shares have generated a snowbal...,0,NaN,I assume these shares have generated a snowbal...
1,These shares have generated a snowball effect,0,NaN,These shares have generated a snowball effect
2,This issue has [justifiably] the public's atte...,0,NaN,This issue has [justifiably] the public's atte...
3,The occasion provides a broad debate in civil ...,0,NaN,The occasion provides a broad debate in civil ...
4,The urgent taking of measurements by the guard...,0,NaN,The urgent taking of measurements by the guard...
...,...,...,...,...
10248,A mayor may belong to the administration of a ...,0,NaN,A mayor may belong to the administration of a ...
10249,ethically it is reprehensible,3,NaN,ethically it is reprehensible
10250,ethically is reprehensible and common sense ad...,0,NaN,ethically is reprehensible and common sense ad...
10251,common sense advises not to do so,0,NaN,common sense advises not to do so


### Splitting the dataset

In [38]:
from datasets import DatasetDict

# 90% train, 10% test+validation
train_test = dataset_hf.train_test_split(test_size=0.1)

# Split the 10% test+validation set in half test, half validation
valid_test = train_test['test'].train_test_split(test_size=0.5)

# gather everyone if you want to have a single DatasetDict
train_valid_test_dataset = DatasetDict({
    'train': train_test['train'],
    'validation': valid_test['train'],
    'test': valid_test['test']
})

### Loading the model and tokenizer

In [50]:
model_name = "bert-base-uncased"

In [51]:
!pip install transformers --quiet

from transformers import AutoTokenizer  # Or BertTokenizer
from transformers import AutoModelForPreTraining  # Or BertForPreTraining for loading pretraining heads
from transformers import AutoModel  # or BertModel, for BERT without pretraining heads
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)
tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=False, padding=True, truncation=True, model_max_len=512)

https://huggingface.co/bert-base-uncased/resolve/main/config.json not found in cache or force_download set to True, downloading to C:\Users\caion\.cache\huggingface\transformers\tmpxmootr5f


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-uncased/resolve/main/config.json in cache at C:\Users\caion/.cache\huggingface\transformers\3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
creating metadata file for C:\Users\caion/.cache\huggingface\transformers\3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at C:\Users\caion/.cache\huggingface\transformers\3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidd

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin in cache at C:\Users\caion/.cache\huggingface\transformers\a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f
creating metadata file for C:\Users\caion/.cache\huggingface\transformers\a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f
loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin from cache at C:\Users\caion/.cache\huggingface\transformers\a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.L

...

storing https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json in cache at C:\Users\caion/.cache\huggingface\transformers\c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
creating metadata file for C:\Users\caion/.cache\huggingface\transformers\c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at C:\Users\caion/.cache\huggingface\transformers\3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt in cache at C:\Users\caion/.cache\huggingface\transformers\45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
creating metadata file for C:\Users\caion/.cache\huggingface\transformers\45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
https://huggingface.co/bert-base-uncased/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to C:\Users\caion\.cache\huggingface\transformers\tmpzfmfuaeh


Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-uncased/resolve/main/tokenizer.json in cache at C:\Users\caion/.cache\huggingface\transformers\534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
creating metadata file for C:\Users\caion/.cache\huggingface\transformers\534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at C:\Users\caion/.cache\huggingface\transformers\45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer.json from cache at C:\Users\caion/.cache\huggingface\transformers\534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc7

## Apply the tokenizer loaded into the text spans

In [41]:
def preprocess_function(sample):
    return tokenizer(sample["tokens"], truncation=True, padding=True)


tokenized_dataset = train_valid_test_dataset.map(preprocess_function, batched=True)
train_valid_test_dataset

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'label', 'article_id'],
        num_rows: 9227
    })
    validation: Dataset({
        features: ['tokens', 'label', 'article_id'],
        num_rows: 513
    })
    test: Dataset({
        features: ['tokens', 'label', 'article_id'],
        num_rows: 513
    })
})

In [52]:
import torch

inputs = tokenizer(train_valid_test_dataset['test'][0]['tokens'], padding=True, truncation=True, return_tensors="pt")

outputs = model(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits)
print(predictions)

tensor([[0.1941, 0.1496, 0.1926, 0.2126, 0.2510]], grad_fn=<SoftmaxBackward0>)


C:\Users\caion\AppData\Local\Temp/ipykernel_15648/361294440.py:6: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = torch.nn.functional.softmax(outputs.logits)


## Fine-tuning

The next step is to fine-tune the model to our training data

In [53]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
from datasets import load_metric
import numpy as np

metric = load_metric("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    output_dir="./results-bert-en-uncased",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# training_args = TrainingArguments("test_trainer", num_train_epochs=1)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [54]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: article_id, tokens. If article_id, tokens are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
C:\Users\caion\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 9227
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1731


Epoch,Training Loss,Validation Loss,Accuracy
1,1.295000,1.262312,0.532164
2,1.249400,1.237766,0.510721
3,1.185600,1.244944,0.524366


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: article_id, tokens. If article_id, tokens are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 513
  Batch size = 16
Saving model checkpoint to ./results-bert-en-uncased\checkpoint-577
Configuration saved in ./results-bert-en-uncased\checkpoint-577\config.json
Model weights saved in ./results-bert-en-uncased\checkpoint-577\pytorch_model.bin
tokenizer config file saved in ./results-bert-en-uncased\checkpoint-577\tokenizer_config.json
Special tokens file saved in ./results-bert-en-uncased\checkpoint-577\special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: article_id, tokens. If article_id, tokens are not expected by `BertForSequenceClas

TrainOutput(global_step=1731, training_loss=1.2275392916903614, metrics={'train_runtime': 8998.9012, 'train_samples_per_second': 3.076, 'train_steps_per_second': 0.192, 'total_flos': 2202864145307880.0, 'train_loss': 1.2275392916903614, 'epoch': 3.0})

In [55]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: article_id, tokens. If article_id, tokens are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 513
  Batch size = 16


{'eval_loss': 1.2377655506134033,
 'eval_accuracy': 0.5107212475633528,
 'eval_runtime': 43.2918,
 'eval_samples_per_second': 11.85,
 'eval_steps_per_second': 0.762,
 'epoch': 3.0}

In [56]:
trainer.save_model()

Saving model checkpoint to ./results-bert-en-uncased
Configuration saved in ./results-bert-en-uncased\config.json
Model weights saved in ./results-bert-en-uncased\pytorch_model.bin
tokenizer config file saved in ./results-bert-en-uncased\tokenizer_config.json
Special tokens file saved in ./results-bert-en-uncased\special_tokens_map.json


### Loading and using a saved model

In [57]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer2 = AutoTokenizer.from_pretrained("./results-bert-en-uncased")
model2 = AutoModelForSequenceClassification.from_pretrained("./results-bert-en-uncased", num_labels=5)

Didn't find file ./results-bert-en-uncased\added_tokens.json. We won't load it.
loading file ./results-bert-en-uncased\vocab.txt
loading file ./results-bert-en-uncased\tokenizer.json
loading file None
loading file ./results-bert-en-uncased\special_tokens_map.json
loading file ./results-bert-en-uncased\tokenizer_config.json
loading configuration file ./results-bert-en-uncased\config.json
Model config BertConfig {
  "_name_or_path": "./results-bert-en-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "lay

In [58]:
from transformers import TextClassificationPipeline

pipe = TextClassificationPipeline(model=model2, tokenizer=tokenizer2) #, return_all_scores=True)

In [59]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

y_pred= []
for p in tokenized_dataset['test']['tokens']:
    ti = tokenizer2(p, return_tensors="pt")
    out = model2(**ti)
    pred = torch.argmax(out.logits)
    y_pred.append(pred)


y_test = tokenized_dataset['test']['label']

print(confusion_matrix(y_test, y_pred))
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred, average='macro'))
print('Recall: ', recall_score(y_test, y_pred, average='macro'))
print('F1: ', f1_score(y_test, y_pred, average='macro'))

[[219  24   0   5   0]
 [ 68  28   0   0   0]
 [ 43   9   0   0   0]
 [ 86   5   0   3   0]
 [ 22   1   0   0   0]]
Accuracy:  0.4873294346978557
Precision:  0.25858208955223877
Recall:  0.24132921528254406
F1:  0.2081731553269862


C:\Users\caion\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
